# Objective: 
Given a protein family dataframe (obtained from UniProt), fetch Rhea identifiers. 

In [10]:
import pandas as pd
import os
import re
import seaborn as sns
import numpy as np

In [2]:
file_in = '../data/annotations/uniprot-proteome_UP000001584.tab'
df = pd.read_csv(file_in, sep = '\t')

In [4]:
df.head(2)

,Entry,Entry name,Status,Protein names,Gene names,Organism,Length,Annotation,Rhea ID,EC number,Catalytic activity,ChEBI,Sequence,Function [CC]
0,L7N6B4,MOAD_MYCTU,reviewed,Molybdopterin synthase sulfur carrier subunit,moaD1 Rv3112 LH57_16990,Mycobacterium tuberculosis (strain ATCC 25618 ...,83,2 out of 5,NaN,NaN,NaN,NaN,MIKVNVLYFGAVREACDETPREEVEVQNGTDVGNLVDQLQQKYPRL...,FUNCTION: Involved in sulfur transfer in the c...
1,P9WJM5,MSRA_MYCTU,reviewed,Peptide methionine sulfoxide reductase MsrA (P...,msrA Rv0137c MTCI5.11c,Mycobacterium tuberculosis (strain ATCC 25618 ...,182,4 out of 5,RHEA:14217; RHEA:19993,1.8.4.11,CATALYTIC ACTIVITY: Reaction=[thioredoxin]-dis...,H2O [CHEBI:15377]; L-methionine [CHEBI:57844];...,MTSNQKAILAGGCFWGLQDLIRNQPGVVSTRVGYSGGNIPNATYRN...,FUNCTION: Has an important function as a repai...


Creating a data frame with only proteins with RHea IDs

In [18]:
df_no_null = df[~df["Rhea ID"].isnull()].copy()

### Fetching Rhea IDs: 

In [19]:
df_no_null.shape

(862, 14)

In [22]:
list_up_entry = []
list_rhea_ids = []
for index, row in df_no_null.iterrows():
    up_entry = row['Entry']
    rhea_ids = row['Rhea ID'].split(';')
    rhea_ids = [rid.split(':')[1] for rid in rhea_ids]
    list_rhea_ids+=rhea_ids
    list_up_entry+=len(rhea_ids)*[up_entry]

df_up_rhea = pd.DataFrame()
df_up_rhea['Entry'] = list_up_entry
df_up_rhea['RheaID'] = list_rhea_ids

In [25]:
df_up_rhea.shape

(1825, 2)

In [23]:
file_out = '../data/up_to_rhea.csv'
df_up_rhea.to_csv(file_out, index = False)

NameError: name 'df_45_l_rhea' is not defined

In [30]:
df_up_rhea.head()
df_up_rhea[df_up_rhea.RheaID == "10868"].Entry.unique()


array(['P9WKC5', 'P9WKC9', 'P9WKA7', 'P9WKB3', 'P9WKA3', 'P9WKC3',
       'P9WKB7', 'P9WKC7', 'P9WQN9', 'P9WQP3', 'P9WQP1', 'P9WKA5',
       'P9WKB1', 'P9WKB5', 'P9WKC1', 'P9WKA9', 'P9WKB9', 'I6Y8R4',
       'O33364'], dtype=object)

In [27]:
df_up_rhea.RheaID.value_counts().head(30)

10868    19
13065    18
47348    17
57888    15
47356    14
12957    13
17989    12
46608    12
45780    11
47364    11
45781    11
20724    10
16105    10
47360     9
47388     9
59388     8
43712     8
43713     8
26526     7
47352     7
39663     7
22508     7
47392     6
30752     6
30751     6
33624     6
33623     6
15421     6
43709     5
10740     5
Name: RheaID, dtype: int64